# सेमांटिक कर्नेल टूल उपयोग उदाहरण

यह दस्तावेज़ सेमांटिक कर्नेल-आधारित टूल बनाने के लिए उपयोग किए गए कोड का अवलोकन और व्याख्या प्रदान करता है, जो Azure AI Search के साथ Retrieval-Augmented Generation (RAG) के लिए एकीकृत होता है। यह उदाहरण दिखाता है कि कैसे एक AI एजेंट बनाया जाए जो Azure AI Search इंडेक्स से यात्रा दस्तावेज़ प्राप्त करता है, उपयोगकर्ता प्रश्नों को सेमांटिक सर्च परिणामों के साथ बढ़ाता है, और विस्तृत यात्रा सिफारिशें स्ट्रीम करता है।


### पैकेज आयात करना  
निम्नलिखित कोड आवश्यक पैकेज आयात करता है:  


In [ ]:
import json
import os

from typing import Annotated

from IPython.display import display, HTML

from dotenv import load_dotenv

from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import SearchIndex, SimpleField, SearchFieldDataType, SearchableField

from openai import AsyncOpenAI

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.contents import FunctionCallContent,FunctionResultContent, StreamingTextContent
from semantic_kernel.functions import kernel_function

### सेमांटिक कर्नेल और एआई सेवा बनाना

एक सेमांटिक कर्नेल इंस्टेंस बनाया जाता है और इसे असिंक्रोनस OpenAI चैट कंप्लीशन सेवा के साथ कॉन्फ़िगर किया जाता है। इस सेवा को कर्नेल में जोड़ा जाता है ताकि प्रतिक्रियाएँ उत्पन्न करने में इसका उपयोग किया जा सके।


In [ ]:
load_dotenv()
# Initialize the asynchronous OpenAI client
client = AsyncOpenAI(
    api_key=os.environ["GITHUB_TOKEN"],
    base_url="https://models.inference.ai.azure.com/"
)

# Create the OpenAI Chat Completion Service
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

### प्रॉम्प्ट प्लगइन को परिभाषित करना

PromptPlugin एक मूल प्लगइन है जो एक फ़ंक्शन को परिभाषित करता है ताकि पुनर्प्राप्ति संदर्भ का उपयोग करके एक संवर्धित प्रॉम्प्ट बनाया जा सके।


In [ ]:
class SearchPlugin:

    def __init__(self, search_client: SearchClient):
        self.search_client = search_client

    @kernel_function(
        name="build_augmented_prompt",
        description="Build an augmented prompt using retrieval context or function results.",
    )
    def build_augmented_prompt(self, query: str, retrieval_context: str) -> str:
        return (
            f"Retrieved Context:\n{retrieval_context}\n\n"
            f"User Query: {query}\n\n"
            "First review the retrieved context, if this does not answer the query, try calling an available plugin functions that might give you an answer. If no context is available, say so."
        )
    
    @kernel_function(
        name="retrieve_documents",
        description="Retrieve documents from the Azure Search service.",
    )
    def get_retrieval_context(self, query: str) -> str:
        results = self.search_client.search(query)
        context_strings = []
        for result in results:
            context_strings.append(f"Document: {result['content']}")
        return "\n\n".join(context_strings) if context_strings else "No results found"

In [ ]:
class WeatherInfoPlugin:
    """A Plugin that provides the average temperature for a travel destination."""

    def __init__(self):
        # Dictionary of destinations and their average temperatures
        self.destination_temperatures = {
            "maldives": "82°F (28°C)",
            "swiss alps": "45°F (7°C)",
            "african safaris": "75°F (24°C)"
        }

    @kernel_function(description="Get the average temperature for a specific travel destination.")
    def get_destination_temperature(self, destination: str) -> Annotated[str, "Returns the average temperature for the destination."]:
        """Get the average temperature for a travel destination."""
        # Normalize the input destination (lowercase)
        normalized_destination = destination.lower()

        # Look up the temperature for the destination
        if normalized_destination in self.destination_temperatures:
            return f"The average temperature in {destination} is {self.destination_temperatures[normalized_destination]}."
        else:
            return f"Sorry, I don't have temperature information for {destination}. Available destinations are: Maldives, Swiss Alps, and African safaris."

## वेक्टर डेटाबेस प्रारंभिकरण

हम Azure AI Search को स्थायी संग्रहण के साथ प्रारंभ करते हैं और उन्नत नमूना दस्तावेज़ जोड़ते हैं। Azure AI Search का उपयोग उन दस्तावेज़ों को संग्रहीत और पुनः प्राप्त करने के लिए किया जाएगा जो सटीक उत्तर उत्पन्न करने के लिए संदर्भ प्रदान करते हैं।


In [ ]:
# Initialize Azure AI Search with persistent storage
search_service_endpoint = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
search_api_key = os.getenv("AZURE_SEARCH_API_KEY")
index_name = "travel-documents"

search_client = SearchClient(
    endpoint=search_service_endpoint,
    index_name=index_name,
    credential=AzureKeyCredential(search_api_key)
)

index_client = SearchIndexClient(
    endpoint=search_service_endpoint,
    credential=AzureKeyCredential(search_api_key)
)

# Define the index schema
fields = [
    SimpleField(name="id", type=SearchFieldDataType.String, key=True),
    SearchableField(name="content", type=SearchFieldDataType.String)
]

index = SearchIndex(name=index_name, fields=fields)

# Check if index already exists if not, create it
try:
    existing_index = index_client.get_index(index_name)
    print(f"Index '{index_name}' already exists, using the existing index.")
except Exception:
    # Create the index if it doesn't exist
    print(f"Creating new index '{index_name}'...")
    index_client.create_index(index)


# Enhanced sample documents
documents = [
    {"id": "1", "content": "Contoso Travel offers luxury vacation packages to exotic destinations worldwide."},
    {"id": "2", "content": "Our premium travel services include personalized itinerary planning and 24/7 concierge support."},
    {"id": "3", "content": "Contoso's travel insurance covers medical emergencies, trip cancellations, and lost baggage."},
    {"id": "4", "content": "Popular destinations include the Maldives, Swiss Alps, and African safaris."},
    {"id": "5", "content": "Contoso Travel provides exclusive access to boutique hotels and private guided tours."}
]

# Add documents to the index
search_client.upload_documents(documents)

In [ ]:
agent = ChatCompletionAgent(
    service=chat_completion_service,
    plugins=[SearchPlugin(search_client=search_client), WeatherInfoPlugin()],
    name="TravelAgent",
    instructions="Answer travel queries using the provided tools and context. If context is provided, do not say 'I have no context for that.'",
)

### स्ट्रीमिंग इनवोकेशन के साथ एजेंट चलाना

मुख्य असिंक्रोनस लूप बातचीत के लिए एक थ्रेड बनाता है और प्रत्येक उपयोगकर्ता इनपुट के लिए, ताकि एजेंट पुनर्प्राप्ति संदर्भ देख सके। उपयोगकर्ता संदेश भी जोड़ा जाता है, और फिर एजेंट को स्ट्रीमिंग के माध्यम से इनवोक किया जाता है। आउटपुट को स्ट्रीमिंग के दौरान प्रिंट किया जाता है।


In [ ]:
async def main():
    thread: ChatHistoryAgentThread | None = None

    user_inputs = [
        "Can you explain Contoso's travel insurance coverage?",
        "What is the average temperature of the Maldives?",
        "What is a good cold destination offered by Contoso and what is it average temperature?",
    ]

    for user_input in user_inputs:
        html_output = (
            f"<div style='margin-bottom:10px'>"
            f"<div style='font-weight:bold'>User:</div>"
            f"<div style='margin-left:20px'>{user_input}</div></div>"
        )

        agent_name = None
        full_response: list[str] = []
        function_calls: list[str] = []

        # Buffer to reconstruct streaming function call
        current_function_name = None
        argument_buffer = ""

        async for response in agent.invoke_stream(
            messages=user_input,
            thread=thread,
        ):
            thread = response.thread
            agent_name = response.name
            content_items = list(response.items)

            for item in content_items:
                if isinstance(item, FunctionCallContent):
                    if item.function_name:
                        current_function_name = item.function_name

                    # Accumulate arguments (streamed in chunks)
                    if isinstance(item.arguments, str):
                        argument_buffer += item.arguments
                elif isinstance(item, FunctionResultContent):
                    # Finalize any pending function call before showing result
                    if current_function_name:
                        formatted_args = argument_buffer.strip()
                        try:
                            parsed_args = json.loads(formatted_args)
                            formatted_args = json.dumps(parsed_args)
                        except Exception:
                            pass  # leave as raw string

                        function_calls.append(f"Calling function: {current_function_name}({formatted_args})")
                        current_function_name = None
                        argument_buffer = ""

                    function_calls.append(f"\nFunction Result:\n\n{item.result}")
                elif isinstance(item, StreamingTextContent) and item.text:
                    full_response.append(item.text)

        if function_calls:
            html_output += (
                "<div style='margin-bottom:10px'>"
                "<details>"
                "<summary style='cursor:pointer; font-weight:bold; color:#0066cc;'>Function Calls (click to expand)</summary>"
                "<div style='margin:10px; padding:10px; background-color:#f8f8f8; "
                "border:1px solid #ddd; border-radius:4px; white-space:pre-wrap; font-size:14px; color:#333;'>"
                f"{chr(10).join(function_calls)}"
                "</div></details></div>"
            )

        html_output += (
            "<div style='margin-bottom:20px'>"
            f"<div style='font-weight:bold'>{agent_name or 'Assistant'}:</div>"
            f"<div style='margin-left:20px; white-space:pre-wrap'>{''.join(full_response)}</div></div><hr>"
        )

        display(HTML(html_output))

await main()


आपका Markdown फ़ाइल का अनुवाद यहां है:

# परिचय

इस दस्तावेज़ में, हम आपको बताएंगे कि कैसे @@INLINE_CODE_0@@ का उपयोग करें। यह टूल आपके काम को आसान और तेज़ बना सकता है।

## आवश्यकताएँ

इससे पहले कि आप शुरू करें, सुनिश्चित करें कि आपके पास निम्नलिखित हैं:

- एक वैध @@INLINE_CODE_1@@ खाता।
- @@INLINE_CODE_2@@ स्थापित है।
- इंटरनेट कनेक्शन।

## चरण-दर-चरण निर्देश

### चरण 1: लॉगिन करें

1. अपने ब्राउज़र में @@INLINE_CODE_3@@ खोलें।
2. अपना उपयोगकर्ता नाम और पासवर्ड दर्ज करें।
3. "लॉगिन" बटन पर क्लिक करें।

[!NOTE] यदि आप अपना पासवर्ड भूल गए हैं, तो "पासवर्ड भूल गए?" लिंक पर क्लिक करें।

### चरण 2: सेटअप करें

1. @@INLINE_CODE_4@@ खोलें।
2. निम्नलिखित कोड को कॉपी करें और @@INLINE_CODE_5@@ में पेस्ट करें:

   ```text
User: 
Can you explain Contoso's travel insurance coverage?

Function Calls (click to expand)

Calling function: retrieve_documents({"query": "Contoso travel insurance coverage"})

Function Result:

Document: Contoso's travel insurance covers medical emergencies, trip cancellations, and lost baggage.

Document: Contoso Travel offers luxury vacation packages to exotic destinations worldwide.

Document: Contoso Travel provides exclusive access to boutique hotels and private guided tours.

Document: Our premium travel services include personalized itinerary planning and 24/7 concierge support.

TravelAgent:

Contoso's travel insurance coverage includes the following:

1. **Medical Emergencies**: Coverage for unforeseen medical issues that may arise while traveling.
2. **Trip Cancellations**: Protection in case you need to cancel your trip for covered reasons.
3. **Lost Baggage**: Compensation for baggage that is lost during your trip.

If you need more specific details about the policy, it would be best to contact Contoso directly or refer to their official documentation.
```

3. "सेव" बटन पर क्लिक करें।

[!TIP] सुनिश्चित करें कि आप कोड को सही तरीके से कॉपी करें।

### चरण 3: परीक्षण करें

1. @@INLINE_CODE_6@@ चलाएं।
2. सुनिश्चित करें कि आउटपुट अपेक्षित है:

   @@CODE_BLOCK_1@@

[!WARNING] यदि आउटपुट सही नहीं है, तो चरण 2 को दोबारा जांचें।

## निष्कर्ष

अब आप सफलतापूर्वक @@INLINE_CODE_0@@ का उपयोग कर सकते हैं। यदि आपको कोई समस्या आती है, तो कृपया हमारे समर्थन टीम से संपर्क करें।



---

**अस्वीकरण**:  
यह दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) का उपयोग करके अनुवादित किया गया है। जबकि हम सटीकता सुनिश्चित करने का प्रयास करते हैं, कृपया ध्यान दें कि स्वचालित अनुवाद में त्रुटियां या अशुद्धियां हो सकती हैं। मूल भाषा में उपलब्ध मूल दस्तावेज़ को प्रामाणिक स्रोत माना जाना चाहिए। महत्वपूर्ण जानकारी के लिए, पेशेवर मानव अनुवाद की सिफारिश की जाती है। इस अनुवाद के उपयोग से उत्पन्न किसी भी गलतफहमी या गलत व्याख्या के लिए हम उत्तरदायी नहीं हैं।
